In [ ]:
import sys;
sys.path.insert(0, '..')

## Chapter 5 Code Snippets and Listings

### Phase oracles (section 5.1.1)

A *phase oracle* rotates the amplitudes of the "good" outcomes by $180^\circ$. Rotating a complex number by $180^\circ$ is the same as multiplying it by -1.

We will start with a classical implementation of phase oracles.


The good outcomes can be specified classically with a predicate that returns `True` for a good outcome and `False` otherwise. For example, if we have a problem with one good outcome, 3, we can define the predicate below:

In [ ]:
predicate = lambda k: True if k == 3 else False

Say we have 8 possible outcomes ($n = 3$). We can use the `predicate` function to list the good outcomes:

In [ ]:
n = 3
print(f'\nGood outcomes: {[k for k in range(2**n) if predicate(k)]}')

We can use this predicate to implement an oracle that takes any state and multiplies the amplitudes of the good states by -1:

In [ ]:
def c_phase_oracle(state, predicate):
    for item in range(len(state)):
        if predicate(item):
            state[item] *= -1

Let's start with an example state with $n = 3$ qubits in a uniform superposition where the magnitudes of all amplitudes are equal.

In [ ]:
from math import sqrt

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

In [ ]:
from util import print_state_table

print_state_table(state)

We can apply the classical phase oracle to this state with the following code:

In [ ]:
c_phase_oracle(state, predicate)

In [ ]:
print('\nState after oracle is applied, changing the direction of good outcomes')
print_state_table(state)

### Bit oracles (section 5.1.2)

A *bit oracle* entangles the good outcomes with an additional qubit, which we will call the "tag bit".

The function `c_bit_oracle` is a classical implementation of a bit oracle:

In [ ]:
def c_bit_oracle(state, predicate):
    N = len(state)
    # Adding a qubit doubles the number of possible outcomes, and therefore the number of amplitudes
    state = state + [0 for _ in range(N)]
    # We find the amplitude corresponding to the outcome with 1 in the tag bit position by adding N to the amplitude index
    for item in range(N):
        if predicate(item):
            state[N + item] = state[item]
            state[item] = 0
    return state

Let's apply this oracle to a state with $n = 3$ qubits, where 3 is the good state:

In [ ]:
predicate = lambda k: True if k == 3 else False

n = 3
state = [1/sqrt(2**n) for _ in range(2**n)]

tag_state = c_bit_oracle(state, predicate)

In [ ]:
print_state_table(tag_state)

We can also apply this oracle to a random state generated using our `generate_state` function:

In [ ]:
from util import generate_state

n = 3
state = generate_state(n, seed=777)

In [ ]:
print_state_table(state)

In [ ]:
state = c_bit_oracle(state, predicate)

In [ ]:
print_state_table(state)

### Creating quantum circuits from building blocks (section 5.2.1)

**Note**: The methods `append` from listing 5.1 and `c_append` from listing 5.2 have been added to the `QuantumCircuit` class in sim_circuit.py.

For example, let's create an example three-qubit register, and a circuit with one X-gate applied to target qubit 0:

In [ ]:
from sim_circuit import *

n = 3
q = QuantumRegister(n)
qc = QuantumCircuit(q)
qc.x(0)

Next, we will use the `uniform` function from chapter 4.
This function creates a circuit for encoding the uniform distribution in a state with `n` qubits.


In [ ]:
def uniform(n):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(len(q)):
        qc.h(q[i])

    return qc

We can apply the circuit defined in `uniform` to our three-qubit register using the `append` method:

In [ ]:
n = 3
uniform_qc = uniform(n)
qc.append(uniform_qc, q) # Apply the circuit to the register q

### Phase oracle (section 5.2.2)

Listing 5.3 Function to create a phase oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [ ]:
from math import pi

    
def is_bit_not_set(m, k):
    return not (m & (1 << k))

def phase_oracle_match(n, items):
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcp(pi, [q[i] for i in range(len(q) - 1)], q[len(q) - 1])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

Let's use this function to create the oracle circuit for $n = 3$ and a single good outcome 3:

In [ ]:
n = 3
items = [3]

oracle_circuit = phase_oracle_match(n, items)

In [ ]:
from util_qiskit import print_circuit

print_circuit(oracle_circuit)

We can create a state in equal superposition and apply the oracle circuit defined above:

In [ ]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])

In [ ]:
qc.append(oracle_circuit, QuantumRegister(n))

In [ ]:
print_state_table(qc.run())

Let's create an oracle for $n = 3$ and associated to good outcomes 1, 3, and 5:

In [ ]:
n = 3
items = [1, 3, 5]

oracle_circuit = phase_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

Let's create a circuit which prepares the state and applies the oracle:

In [ ]:
q = QuantumRegister(n)
qc = QuantumCircuit(q)

for i in range(n):
    qc.h(q[i])
    
qc.append(oracle_circuit, QuantumRegister(n))    

In [ ]:
print_state_table(qc.run())

### Bit oracle (section 5.2.3)

Listing 5.4 Function to create a bit oracle quantum circuit for a given number of qubits `n` and a set of good outcomes `items`

In [ ]:
def bit_oracle_match(n, items):
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)

    for m in items:
        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])

        qc.mcx([q[i] for i in range(len(q))], a[0])

        for i in range(n):
            if is_bit_not_set(m, i):
                qc.x(q[i])
    return qc

Let's create the bit oracle circuit and apply it to our familiar example, where a state with $n = 3$ qubits is prepared using Hadamard gates and the good item is 3.

In [ ]:
n = 3
items = [3]

oracle_circuit = bit_oracle_match(n, items)

q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n + 1)) # <1>

In [ ]:
print_circuit(oracle_circuit)

In [ ]:
print_state_table(qc.run())

Next, let's create a bit oracle for the same state with three good outcomes:

In [ ]:
n = 3
items = [1, 3, 5]

oracle_circuit = bit_oracle_match(n, items)

q = QuantumRegister(n)
a = QuantumRegister(1)
qc = QuantumCircuit(q, a)

for i in range(n):
    qc.h(q[i])

qc.append(oracle_circuit, QuantumRegister(n+1))

In [ ]:
print_circuit(oracle_circuit)

In [ ]:
print_state_table(qc.run())

### From a phase oracle to a bit oracle (5.3.1)

If we have a phase oracle circuit defined using the `QuantumCircuit` class in our simulator, we can use it to create a bit oracle circuit using the function below:

In [ ]:
def phase_to_bit_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs) # Get the number of qubits used for the phase oracle
    q = QuantumRegister(n)
    a = QuantumRegister(1)
    qc = QuantumCircuit(q, a)
    qc.h(a[0])
    qc.c_append(oracle_circuit, a[0], q) # Apply the phase oracle circuit controlled on the ancilla qubit
    qc.h(a[0])

    return qc

For example, let's create the phase oracle circuit for $n = 3$ qubits and good outcomes 1, 3, and 5:

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = phase_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

Let's use the `generate_state` function to generate a random state with $n = 3$ qubits and an ancilla qubit:

In [ ]:
state = generate_state(n, seed=777) + [0 for _ in range(2**n)]

In [ ]:
print_state_table(state)

**Note:** In this chapter, we add the method `initialize` to the `QuantumCircuit` class in sim_circuit.py, which allows us to write the state in a `QuantumCircuit` class instance.

In [ ]:
q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(phase_to_bit_oracle(oracle_circuit), QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

### From a bit oracle to a phase oracle (section 5.3.2)

We can use the following function to create a circuit which will act as a phase oracle, where the parameter `oracle_circuit` is a bit oracle:

In [ ]:
def bit_to_phase_oracle(oracle_circuit):
    n = sum(oracle_circuit.regs)
    q = QuantumRegister(n)
    qc = QuantumCircuit(q)
    qc.append(oracle_circuit, q)
    qc.p(pi, q[len(q)-1])
    qc.append(oracle_circuit, q)

    return qc

Let's create the bit oracle circuit for our example problem where $n = 3$ qubits and good outcomes are 1, 3, and 5:

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

In [ ]:
print_circuit(oracle_circuit)

Let's generate a random state and use the bit oracle above to create a phase oracle:

In [ ]:
n = 3
items = [1, 3, 5]
oracle_circuit = bit_oracle_match(n, items)

state = generate_state(n, seed=777) + [0 for _ in range(2**n)]

q = QuantumRegister(n)
a = QuantumRegister(1)

qc = QuantumCircuit(q, a)
qc.initialize(state.copy())

qc.append(bit_to_phase_oracle(oracle_circuit), QuantumRegister(n+1))

In [ ]:
print_state_table(qc.run())

### Fibonacci numbers and the golden ratio with good outcomes (section 5.4)

We can compute the $n^{th}$ number in the Fibonacci sequence (denoted by $F_n$) using the recursive Python function below:

In [ ]:
def recursive_fib(n):
    if n <= 1:
        return n
    else:
        return recursive_fib(n - 1) + recursive_fib(n - 2)

We can use the recursive function to create a list of the first 10 Fibonacci numbers:

In [ ]:
[recursive_fib(n) for n in range(10)]

We can create a circuit that identifies the good outcomes and makes the bad outcomes impossible. The function `fib_circuit` creates this circuit for a given number of qubits $n > 0$:

In [ ]:
from math import asin

def fib_circuit(n):
    theta = 2*asin((sqrt(5) - 1)/2)

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for i in range(n):
        qc.ry(theta, q[i])

    for i in range(n - 1):
        qc.cry(-theta, q[i], q[i + 1])

    return qc

Let's create the circuit for one qubit:

In [ ]:
qc = fib_circuit(1)

In [ ]:
print_state_table(qc.run())

For a given number of qubits $n$ we can see that:

* There are $F_{n+1}$ good outcomes with the first binary digit 0 (top half of the state table), and the amplitudes corresponding to these outcomes are all equal.
* There are $F_n$ good outcomes with the first binary digit 1 (bottom half of the state table), and the amplitudes corresponding to these outcomes are all equal.

We can check that the ratio of the probability of a good outcome that starts with 0 and that of a good outcome that starts with 1 is the golden ratio:

In [ ]:
from util import is_close

qc = fib_circuit(2)
state = qc.run()

assert is_close(abs(state[0])**2/abs(state[2])**2, (1+sqrt(5))/2) # <1>
assert is_close(abs(state[1])**2/abs(state[2])**2, (1+sqrt(5))/2) # <2>

In [ ]:
print_state_table(state)

3 qubit example:

In [ ]:
qc = fib_circuit(3)
print_state_table(qc.run())